# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [117]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import key_dict
gkey = key_dict['maps']
gmaps.configure(api_key=gkey)

# Output File (CSV)
output_data_file = "../output_data/cities.csv"

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [56]:
df=pd.read_csv(output_data_file)
df.head()

,Cities,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bambanglipuro,-7.9500,110.2833,78.49,82,99,3.89,ID,2021-03-20 23:39:08
1,guozhen,34.3659,107.3590,42.58,27,100,1.43,CN,2021-03-20 23:39:09
2,mezen,65.8522,44.2400,13.12,96,94,9.35,RU,2021-03-20 23:39:09
3,lebu,-37.6167,-73.6500,56.64,81,4,6.46,CL,2021-03-20 23:39:10
4,vaini,-21.2000,-175.2000,86.00,74,40,6.91,TO,2021-03-20 23:39:10


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
locations=df[["Lat", "Lng"]]
humidity=df['Humidity']
figure_layout = {
    'width': '750px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, center = (30,0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [99]:
ideal_df=df[df["Max Temp"] > 75]
ideal_df=ideal_df[ideal_df["Max Temp"] < 90]
ideal_df=ideal_df[ideal_df["Wind Speed"] < 10]
ideal_df=ideal_df[ideal_df["Cloudiness"] < 10]
ideal_df=ideal_df[ideal_df["Humidity"] > 30]
ideal_df=ideal_df[ideal_df["Humidity"] < 80]
print(ideal_df.describe())
ideal_df

             Lat         Lng   Max Temp   Humidity  Cloudiness  Wind Speed
count  15.000000   15.000000  15.000000  15.000000   15.000000   15.000000
mean    4.903573   -3.193307  80.614667  62.266667    2.866667    6.111333
std    13.741205   83.398465   3.756216  13.956395    3.356586    2.605146
min   -23.120300 -151.750000  75.200000  37.000000    0.000000    1.880000
25%    -4.680300  -81.164550  76.895000  58.000000    0.000000    4.610000
50%     9.494200   37.332100  80.600000  68.000000    1.000000    5.120000
75%    13.186100   52.452300  83.030000  71.000000    5.500000    8.970000
max    23.613900  122.671100  86.000000  78.000000    9.000000    9.530000


,Cities,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,rikitea,-23.1203,-134.9692,80.08,71,4,4.72,PF,2021-03-20 23:35:19
56,faanui,-16.4833,-151.7500,83.12,68,1,7.18,PF,2021-03-20 23:39:33
71,mattru,7.6244,-11.8332,75.65,71,9,1.88,SL,2021-03-20 23:39:40
184,salalah,17.0151,54.0924,75.20,57,0,4.61,OM,2021-03-20 23:40:30
207,bandarbeyla,9.4942,50.8122,76.30,78,3,9.06,SO,2021-03-20 23:40:39
293,el alto,-4.2714,-81.2147,80.60,69,0,9.22,PE,2021-03-20 23:41:13
296,dingle,10.9995,122.6711,82.54,78,7,8.88,PH,2021-03-20 23:41:15
372,sawakin,19.1059,37.3321,75.72,59,0,4.63,SD,2021-03-20 23:41:49
373,mitsamiouli,-11.3847,43.2844,82.94,71,8,5.12,KM,2021-03-20 23:41:50
416,muscat,23.6139,58.5922,84.20,37,0,2.30,OM,2021-03-20 23:42:09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [161]:
hotel_df=ideal_df[['Cities','Country','Lat','Lng']]
hotel_df['Hotel Name']=''

hotel_df=hotel_df.rename(columns={"Cities":"City"})
hotel_df.head()

C:\Users\Pearl\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
27,rikitea,PF,-23.1203,-134.9692,
56,faanui,PF,-16.4833,-151.7500,
71,mattru,SL,7.6244,-11.8332,
184,salalah,OM,17.0151,54.0924,
207,bandarbeyla,SO,9.4942,50.8122,


In [162]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#target_coordinates= []
params={
#    "location": target_coordinates,
    "keyword": "hotel",
    "radius": 5000,
#    "type": "lodging",
    "key": gkey ,
#    "rankby": "DISTANCE"
}


In [163]:
for index,row in hotel_df.iterrows():
    params["location"]= f"{row['Lat']},{row['Lng']}"
    resp = requests.get(url, params=params).json()
    try: 
        results=resp['results'][0]
    
        hotel_df.loc[index,"Hotel Name"] = results["name"]
    except:
        hotel_df.drop(index, inplace=True)
        print(f"No hotel not found. Drop {row['City']}")

    #pprint(resp)
    #print(params)
    #print(params["location"])
    

No hotel not found. Drop dingle


In [165]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
27,rikitea,PF,-23.1203,-134.9692,Pension Bianca & Benoit
56,faanui,PF,-16.4833,-151.7500,InterContinental Bora Bora Resort Thalasso Spa
71,mattru,SL,7.6244,-11.8332,World Vision
184,salalah,OM,17.0151,54.0924,Star Emirates garden villas
207,bandarbeyla,SO,9.4942,50.8122,JABIR HOTEL
293,el alto,PE,-4.2714,-81.2147,"Hotel ""El Merlin"" Cabo Blanco"
372,sawakin,SD,19.1059,37.3321,فندق عرب بلازا
373,mitsamiouli,KM,-11.3847,43.2844,Al Camar Lodge
416,muscat,OM,23.6139,58.5922,"Al Bustan Palace, A Ritz-Carlton Hotel"
422,acajutla,SV,13.5928,-89.8275,Villavela Hotel Boutique


In [168]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [170]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…